In [125]:
from math import log2
#the parameters of the buildTree function take as parameters:
    #D which is the input Data (Records)
    #dic which is the dictionnary that correspands each attribute to an index (example dic['A']=0)
    #Attributes contains the values that can be taken by each attribute (exp Attributes[0] contains 
                                                                    #the values that can be taken by the attribute 'A')
    #A is a list of all the attributes
    #node is the position of the node in the tree
    #lvl is the level of the corresponding node
    
def BuildTree(D,dic,Attributes,A,minNum,node,lvl,d,tree):
    # tree[node] = [-1 if not leaf and class if leaf , level, Feature , GINI]
    GINI_split = 0 # the GINI of The corresponding node
    ma = 0 #used to maximize information gain
    
    res = '';
    
    splitAtt = 0; #the best attribute we will chose to split the tree
    
    if (len(A)==0): # if we finished splitting the tree then the node will be a leaf
                    # and will take all the records in the same class
        tree[node] = (D[0][len(D[0])-1],lvl,"",0)
        return;
    if len(D)<minNum: # if the number of records <minNum the the node will be a leaf take the class d (default)
            tree[node] = (d,lvl,"",0)
            return
    for a in A: # iterate attributes to find the best split
        Att=dic[a] # the index of the corresponding attribute in the input data
        test = True # to check if all records have the same class
        clas = D[0][len(D[0])-1] # class of the first record
        L = Attributes[Att] # the possible values that the attribute can take
        C1 = [0 for co in range(len(L))] # holds for each possible value the number of records having class 0
        C2 = [0 for co in range(len(L))] # holds for each possible value the number of records having class 1
        for rec in D:
            if rec[len(D[0])-1]==0:
                C1[rec[Att]]+=1
            else:
                C2[rec[Att]]+=1

            if (rec[len(D[0])-1]!=clas):
                test = False
        
        if (test): # if all records have same class the node is a leaf with the same class
            tree[node] = (clas,lvl,"",0)
            return

        
        for spli in range(1,len(L)): # spli is the index of the split
                spl1=L[0:spli] # spl1 values to the left
                spl2=L[spli:]  # spl2 values to the right
                
                c01 = sum(C1) # number of records having class =0 in the node
                c02 = sum(C2) # number of records having class =1 in the node
                c11 = sum(C1[0:spli]) # number of records having class =0 in the left child
                c12 = sum(C2[0:spli]) # number of records having class =1 in the left child
                c21 = sum(C1[spli:]) # number of records having class =0 in the right child
                c22 = sum(C2[spli:]) # number of records having class =1 in the right child
                
               
                EntropyP = 0;Entropy1=0;Entropy2=0; # Entropy1 is the entropy of the left child and Entropy2 of the right child
                                                    # EntropyP is the entropy of the parent
                
                if (c01+c02)!=0 and (c01/(c01+c02))!=0: # for no log2(0)
                    EntropyP-= (c01/(c01+c02)) * log2((c01/(c01+c02))) 
                if (c01+c02)!=0 and (c02/(c01+c02))!=0:
                    EntropyP-= (c02/(c01+c02)) * log2((c02/(c01+c02)))
                if (c11+c12)!=0 and (c11/(c11+c12))!=0:
                    Entropy1 = -1 * (c11/(c11+c12)) * log2((c11/(c11+c12))) 
                if (c11+c12)!=0 and (c12/(c11+c12))!=0:
                    Entropy1-= (c12/(c11+c12)) * log2((c12/(c11+c12)))
                if (c21+c22)!=0 and (c22/(c21+c22))!=0:
                    Entropy2 = -1 * (c22/(c21+c22)) * log2((c22/(c21+c22))) 
                if (c21+c22)!=0 and (c21/(c21+c22))!=0:
                    Entropy2-= (c21/(c21+c22)) * log2((c21/(c21+c22)))
                Entropy_split = EntropyP - (Entropy1 * ((c11+c12)/(c11+c12+c22+c21)) + Entropy2*((c21+c22)/(c11+c12+c22+c21)))
                if (Entropy_split>=ma): #maximizing the information gain
                    ma = Entropy_split
                    res = spli
                    splitAtt = a
                    GINI1=0;GINI2=0
                    if (c11+c12)!=0:
                        GINI1=1-(c11/(c11+c12))**2-(c12/(c11+c12))**2
                    if (c21+c22)!=0:
                        GINI2=1-(c22/(c21+c22))**2-(c21/(c21+c22))**2
                    GINI_split = GINI1 * ((c11+c12)/(c11+c12+c22+c21)) + GINI2*((c21+c22)/(c11+c12+c22+c21))

    right = [] #records to the right
    left=[]    #records to the left
    
    L=Attributes[dic[splitAtt]] #index of the chosen attribute that maximizes info gain
    
    for i in D:
        if i[dic[splitAtt]] in L[0:res]:
            left.append(i)
        else:
            right.append(i)
    
    st = splitAtt # st is the string of the feature example 'A 0 1'
   
    for ss in L[0:res]:
        st+=" "
        st+=str(ss) 
    tree[node]=(-1,lvl,st,GINI_split) #finally setting the node to the chosen values
    LL = A.copy()
    LL.pop(LL.index(splitAtt)) # deleting the used attribute from the set of attributes
    
    
    
    
    #recursive call for right and left child
    BuildTree(left,dic,Attributes,LL,minNum,node*2+1,lvl+1,d,tree) 
    BuildTree(right,dic,Attributes,LL,minNum,node*2+2,lvl+1,d,tree)
    
              

In [126]:
def BuildDecisionTree(D,dic,Attributes,A,minNum,node,lvl,d):
    tree=[() for i in range(2**(len(Attributes)+1))] # the resulting tree passed as global var to BuildTree
    # tree[node] = [-1 if not leaf and class if leaf , level, Feature , GINI]
    BuildTree(D,dic,Attributes,A,minNum,node,lvl,d,tree)
    return tree

    

In [127]:
def printTree(tree):
    #print tree BFS
    
    q=[]
    q.append(0)
    while(len(q)>0):
        node = q.pop(0)
        t = tree[node]
        
        if (node==0):
            if (t[0]==-1): # if node is root but not leaf
                print("Root")
                print("Level",t[1])
                print("Feature",t[2] )
                print("Gini",t[3])
                q.append(node*2+1)
                q.append(node*2+2)
            else : # if tree has only one node
                print("Root/leaf")
                print("Level",t[1])
                
                print("Gini",t[3])
                
        else:
            if (t[0]==-1): # if node is not leaf
                print("Intermediate")
                print("Level",t[1])
                print("Feature :",t[2] )
                print("Gini",t[3])
                q.append(node*2+1)
                q.append(node*2+2)
            else: # if node is leaf
                print("Leaf")
                print("Level",t[1])
                print("Gini",t[3])


    

In [128]:
#parameters are :
    # data is the input data in the format list[list[int]]
    # dic is the dictionnary linking each attribute to an index
    # tree is the resulting tree from BuildDecisionTree (heap Tree)
    # alpha is float
def error(data,dic,tree,alpha):
    num_leaves = 0 # number of leaves in tree
    num_errors=0   # number of wrong predictions for the data set
    for i in range(len(tree)): # counting the leaves
        if len(tree[i])==0:
            continue
        if  tree[i][0]!=-1:
            num_leaves+=1
            
    testTree = [[] for k in range(len(tree))] # testTree is the tree containing the records at each node
    
    testTree[0] = data # the root has all the records 
    
    for i in range(len(tree)):
        if len(tree[i])==0:
            continue
        node = tree[i]
        if (node[0]==-1): ## node is not leaf
            condition = node[2].split(" ") # node 2 is like "A 0 1 2"
            At = condition[0] ## tested attribute
            values = [int(x) for x in condition[1:]] 
            # if the value of the tested data of the corresponding attribute is in values then we go left else we go right
            for prod in testTree[i]:
                if prod[dic[At]] in values:
                    testTree[i*2+1].append(prod)
                else :
                    testTree[i*2+2].append(prod)
        
        else : # if node is a leaf
            
            for prod in testTree[i]: 
                # for each record in the node of testTree we check if its class is equal to the predicted class
                if prod[len(prod)-1]!=node[0]:
                    num_errors+=1 # if class[prod] is not equal to the predicted class then we add 1 to error
    genErr = num_errors+alpha*num_leaves
    return  genErr 

In [129]:
#parameters are :
    # data is the input data in the format list[list[int]]
    # dic is the dictionnary linking each attribute to an index
    # tree is the resulting tree from BuildDecisionTree (heap Tree)
    # alpha is float
    # minNum is int 
    # d is the default class (binary)
def prune(data,dic,tree,alpha, minNum, d):
 
    testTree = [ [] for k in range(len(tree))] # same as with error function we built the test tree
    testTree[0] = data
    for i in range(len(tree)): 
        if len(tree[i])==0:
            continue
        node = tree[i]
        if (node[2]!=""): ## node is not leaf
            condition = node[2].split(" ")
            At = condition[0] ## tested attribute
            values = [int(x) for x in condition[1:]]
            for prod in testTree[i]:
                if prod[dic[At]] in values:
                    testTree[i*2+1].append(prod)
                else :
                    testTree[i*2+2].append(prod)
    
    pruned = [tt for tt in tree] # pruned is the result of this function it starts the same as tree
    err = error(data,dic,tree,alpha) # compute the generalization error of the tree 
    
    for i in range(len(tree)-1,-1,-1): # starting from the last node (MaxLvl downto 0)
        if len(pruned[i])==0:
            continue
        node = pruned[i] # node of the pruned tree (starting from last node)
        temp = [xx for xx in pruned] # temp is the temporary tree where we try to prune and see the result
        if len(node)>0 and node[0]==-1: #if node is not leaf
            records = testTree[i] # the records contained in the corresponding node (=num records left child+num records right child)
            if len(records)<minNum: # if number of records <minNum we prune with the default class d
                temp[i] = (d,tree[i][1],"",0) # the node becomes a leaf with class d
                temp[2*i+1]=[] #delete left and right child
                temp[2*i+2]=[]
                tempErr = error(data,dic,temp,alpha) # compute gen error of the temp tree
                if (tempErr<err): # minimizing the err
                    pruned = temp # if tempErr < err then we prune and update err
                    err = tempErr
            else:
                cla1 = 0;cla2 = 0;
                # cla1 number of records having the class 0 and cla2 number of records having the class 1
                for record in records: #counting
                    if record[len(record)-1]==0:
                        cla1+=1
                    else :
                        cla2+=1
                        
                if cla1>cla2: # then the node is a leaf with class 0
                    temp[i] = (0,tree[i][1],"",0)
                    temp[2*i+1]=[]
                    temp[2*i+2]=[]
                    tempErr = error(data,dic,temp,alpha)# compute gen error of the temp tree
                    if (tempErr<err):# if tempErr < err then we prune and update err
                        pruned = temp
                        err = tempErr
                    
                else : # then the node is a leaf with class 1
                    temp[i] = (1,tree[i][1],"",0)
                    temp[2*i+1]=()
                    temp[2*i+2]=()
                    tempErr = error(data,dic,temp,alpha)# compute gen error of the temp tree
                    if (tempErr<err):# if tempErr < err then we prune and update err
                        pruned = temp
                        err = tempErr
    print("the generalization error of the pruned tree is = ",err) # printing the gen error
    
    return pruned 


            
            

In [131]:
f = open("text.txt",'r').read().strip().split("\n")
A=['A','B','C','D'] ## names of attributes we suppose that this is the same order as the input file.txt

Attributes = [[0,1,2],[0,1,2],[0,1],[0,1]] # values taken by attributes

dic={} # correspands each attribute to an index
inde = 0;
## indexing the attributes
for a in A:
    dic[a]=inde
    inde+=1

lis = [i.split(" ") for i in f] # in format List[List[string]]
ll=[] # the input converted to List[List[int]]

for i in lis:
    temp=[]
    for j in i:
        temp.append(int(j))
    ll.append(temp)

tree = BuildDecisionTree(ll,dic,Attributes,A,0,0,1,0)

print("-------------------------------------------------------------")
print ("printing the decision tree")
print("-------------------------------------------------------------")
printTree(tree)
print("-------------------------------------------------------------")
print("the generalisation error of this tree is : ")
print(error(ll,dic,tree,0.5))
print("-------------------------------------------------------------")
print("pruning the tree")
print("-------------------------------------------------------------")
printTree(prune(ll,dic,tree,0.5, 0, 0))

-------------------------------------------------------------
printing the decision tree
-------------------------------------------------------------
Root
Level 1
Feature A 0 1
Gini 0.35714285714285715
Intermediate
Level 2
Feature : C 0
Gini 0.31999999999999984
Leaf
Level 2
Gini 0
Intermediate
Level 3
Feature : D 0
Gini 0.2
Intermediate
Level 3
Feature : D 0
Gini 0.26666666666666666
Leaf
Level 4
Gini 0
Intermediate
Level 4
Feature : B 0 1
Gini 0.0
Intermediate
Level 4
Feature : B 0 1
Gini 0.3333333333333333
Leaf
Level 4
Gini 0
Leaf
Level 5
Gini 0
Leaf
Level 5
Gini 0
Leaf
Level 5
Gini 0
Leaf
Level 5
Gini 0
-------------------------------------------------------------
the generalisation error of this tree is : 
4.5
-------------------------------------------------------------
pruning the tree
-------------------------------------------------------------
the generalization error of the pruned tree is =  3.5
Root
Level 1
Feature A 0 1
Gini 0.35714285714285715
Intermediate
Level 2
Feature 